## Datasets and setup 
Our datasets (`\ds1` and `\ds2`) are somewhat complex but both share the same structure:
- `\packages` contains the packages to analyse and to generate SBOMs from
- `\sbom` contains the generated SBOMs generated by each tool for each package

### ds1
Dataset n1 (ds1) is "home-made" following Cofano et al. Cofano had no original gruth truth, so no ground truth directory is present.

### ds2
Dataset n2 (ds2) is a copy from Jia et al's dataset and thus directly allows us to reuse their data for comparison without having to reexcute each tool on each packagee. `\deptree_gt` contains the ground truth as json files for each package.